In [1]:
# importation des modules

import os
import csv
import json
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
# extraction des tweets

tweet_set = "mpt"
path_to_tweets = f'../../../Données/all-{tweet_set}-tweets.csv'

df = pd.read_csv(path_to_tweets, sep=',')
sorted_tweets = df.sort_values(by="average_hash")

In [13]:
# fonction d'écriture des liens au format csv

def dict_to_csv(dict):
    edges = "source;target;weight;sha1;average_hash\n"
    for key, value in dict.items():
            user_1, user_2 = key.split('-')
            edges += f"{user_1};{user_2};{value['weight']};{value['sha1']};{value['average_hash']}\n"
    return edges

In [14]:
# fonction d'insertion d'un lien dans le dictionnaire

def add_edge(dict, key, sha1, ahash):
    # si le couple de comptes n'est pas référencé dans le dictionnaire, créer le lien
    if key not in dict:
        dict[key] = {"weight": 0, "sha1": sha1, "average_hash": ahash}
    # si le couple de comptes est déjà référencé dans le dictionnaire, augmenter le poids de leur lien
    dict[key]["weight"] += 1

In [15]:
# stocker les ahash de la première ligne pour initialiser
current_ahash = sorted_tweets["average_hash"][0]

# on stock dans ce tableau des triplets de type (username, sha1, tweet_text)
# il permet de comparer l'utilisateur actuel de la boucle (user_name) avec tous les autres utilisateurs ayant partagé le même averagehash
current_users = []

# initialisation du dictionnaire de liens de copublication
copublishing_links = {}

with tqdm(total=sorted_tweets.shape[0]) as pbar:
    # pour chaque ligne du csv
    for index, row in sorted_tweets.iterrows():
        
        user_name = row['user_name']
        ahash = row['average_hash']
        sha1 = row['sha1']
        text = row['text']

        # si le averagehash est le même que celui d'avant
        if ahash == current_ahash:
            # on va checker tous les users d'avant qui avaient le même average hash
            for current_user, current_sha1, current_text in current_users:
                # on vérifie que ça n'est pas le même user, ni le même sha1
                if current_user != user_name and not current_text.startswith("RT @") and not text.startswith("RT @"):
                    key = "-".join(sorted([current_user, user_name]))
                    # on créé un lien de partage
                    add_edge(copublishing_links, key, sha1, ahash)
        # sinon on remet à 0 la liste des utilisateurs du averagehash
        else:
            current_users = []
            current_ahash = ahash
        # dans tous les cas on ajoute le user qu'on est en train de traiter avec son sha1 à la liste des users à comparer
        current_users.append([user_name, sha1, text])

        pbar.update(1)


  0%|          | 0/3666 [00:00<?, ?it/s]

In [16]:
# Écrire le .csv

with open(f"strict-copublishing-{tweet_set}.csv", 'w', encoding='utf-8') as f:
    f.write(dict_to_csv(copublishing_links))